In [86]:
# Combine all the team_records into one dataframe
# This is the first step in the process of creating a singlular dataframe of all team information, FPI data, team records, coaches and transfers
# Completed by Nathan Grey

import pandas as pd

df_2024 = pd.read_csv('./data/raw/team_info/2024_team_records.csv')
df_2023 = pd.read_csv('./data/raw/team_info/2023_team_records.csv')
df_2022 = pd.read_csv('./data/raw/team_info/2022_team_records.csv')

df_team_records_22_to_24 = pd.concat([df_2022, df_2023, df_2024])

# Drop every team other than fbs and fcs
df_team_records_22_to_24 = df_team_records_22_to_24[df_team_records_22_to_24["Classification"].isin(["fbs", "fcs"])]

# Drop all columns except Year, TeamId, Team, Classification, Conference, 
# Division, ExpectedWins, Total Games, Total Wins, Total Losses, Total Ties
df_team_records_22_to_24 = df_team_records_22_to_24[["Year", "TeamId", "Team", "Classification", "Conference", "Division", 
                        "ExpectedWins", "Total Games", "Total Wins", "Total Losses", "Total Ties"]]

df_team_records_22_to_24.head(5)

,Year,TeamId,Team,Classification,Conference,Division,ExpectedWins,Total Games,Total Wins,Total Losses,Total Ties
0,2022,2754,Youngstown State,fcs,MVFC,NaN,5.316334,11,7,4,0
1,2022,43,Yale,fcs,Ivy,NaN,6.001470,10,8,2,0
2,2022,2751,Wyoming,fbs,Mountain West,Mountain,6.018762,13,7,6,0
6,2022,2747,Wofford,fcs,Southern,NaN,2.684401,11,3,8,0
17,2022,275,Wisconsin,fbs,Big Ten,West,8.817187,13,7,6,0


In [87]:
# Combine all FPI dataframes 
# We want to combine all the FPI dataframes into one dataframe we would also like to only get information from the beginning of the season to the end of the season
# Completed by Nathan Grey

import pandas as pd

# List of FPI dataframes
df_fpi_2022 = pd.read_csv('./data/raw/team_info/2022_FPI.csv')
df_fpi_2023 = pd.read_csv('./data/raw/team_info/2023_FPI.csv')
df_fpi_2024 = pd.read_csv('./data/raw/team_info/2024_FPI.csv')

# Combine all FPI dataframes
df_fpi_22_to_24 = pd.concat([df_fpi_2022, df_fpi_2023, df_fpi_2024])

# Drop all columns except Year, Team, FPI
df_fpi_22_to_24 = df_fpi_22_to_24[["Year", "Team", "Fpi"]]

# Combine FPI data with team records using both Year and Team for merging
df_team_records_22_to_24 = pd.merge(
    df_team_records_22_to_24, 
    df_fpi_22_to_24, 
    on=['Year', 'Team'], 
    how='left'
)

df_team_records_22_to_24.head(5)


,Year,TeamId,Team,Classification,Conference,Division,ExpectedWins,Total Games,Total Wins,Total Losses,Total Ties,Fpi
0,2022,2754,Youngstown State,fcs,MVFC,NaN,5.316334,11,7,4,0,NaN
1,2022,43,Yale,fcs,Ivy,NaN,6.001470,10,8,2,0,NaN
2,2022,2751,Wyoming,fbs,Mountain West,Mountain,6.018762,13,7,6,0,-9.340
3,2022,2747,Wofford,fcs,Southern,NaN,2.684401,11,3,8,0,NaN
4,2022,275,Wisconsin,fbs,Big Ten,West,8.817187,13,7,6,0,8.148


In [ ]:
# We want to add coach information to the team records dataframe
# Completed by Nathan Grey

import pandas as pd

# Read in coach information
df_coach_info = pd.read_csv('./data/raw/team_info/coaches.csv')

# Change the column School to Team and combine FirstName and LastName to create a single Coach column
df_coach_info = df_coach_info.rename(columns={'School': 'Team'})
df_coach_info['Coach'] = df_coach_info['FirstName'].fillna('') + ' ' + df_coach_info['LastName'].fillna('')
df_coach_info['Coach'] = df_coach_info['Coach'].str.strip()

# Remove any duplicate combinations of Year, Team, Coach (in case of multiple entries per year/team)
df_coach_info = df_coach_info[["Year", "Team", "Coach"]].drop_duplicates(subset=['Year', 'Team'])

# Add an empty row for every team for year 2025

# Get a list of all teams (and their TeamId if present) from the most recent year (2024)
latest_year = "2024"
teams_2024 = df_team_records_22_to_24[df_team_records_22_to_24['Year'] == latest_year][['TeamId', 'Team']].drop_duplicates()

# Build an empty DataFrame for 2025 (copying all columns from df_team_records_22_to_24)
columns = df_team_records_22_to_24.columns

# Create new rows with Year 2025 and empty values for every other column except Team and TeamId
df_2025 = pd.DataFrame({
    'Year': '2025',
    'TeamId': teams_2024['TeamId'],
    'Team': teams_2024['Team']
})

# Add any other columns that are present in the main dataframe but not in the above
for col in columns:
    if col not in df_2025.columns:
        df_2025[col] = pd.NA  # or could use numpy.nan if preferred

# Reorder columns to match
df_2025 = df_2025[columns]

# Append to the main dataframe
df_team_records_22_to_24 = pd.concat([df_team_records_22_to_24, df_2025], ignore_index=True)

# Make sure 'Year' in both dataframes are of the same type
df_team_records_22_to_24['Year'] = df_team_records_22_to_24['Year'].astype(str)
df_coach_info['Year'] = df_coach_info['Year'].astype(str)

# Merge coach info, overwriting any old Coach column, ensuring only a single coach column appears
df_team_records_22_to_24 = df_team_records_22_to_24.drop(columns=[col for col in df_team_records_22_to_24.columns if col.lower() == 'coach'], errors='ignore')
df_team_records_22_to_24 = df_team_records_22_to_24.merge(
    df_coach_info,
    on=['Year', 'Team'],
    how='left'
)

# Ensure rows are ordered by team and year before looking ahead to the next season
df_team_records_22_to_24 = df_team_records_22_to_24.sort_values(['Team', 'Year']).reset_index(drop=True)

# Add a column for coaching change that checks to see if the coach in the next year is different from the coach in the current year
df_team_records_22_to_24['CoachingChange'] = df_team_records_22_to_24.groupby('Team')['Coach'].transform(
    lambda s: s.fillna('').ne(s.shift(-1).fillna('')) & s.shift(-1).notna()
)

# Now lets drop all 2025 rows 
# We drop the 2025 rows because we are currently in the 2025 season and we are only looking for 2022-2024 data
# We had to add the 2025 rows to be able to add the 2025 coach information and add the coaching change column
df_team_records_22_to_24 = df_team_records_22_to_24[df_team_records_22_to_24['Year'] != '2025']


/var/folders/zg/x_l2tpj15rgctrj55z8jjgvh0000gn/T/ipykernel_25855/319920514.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_team_records_22_to_24 = pd.concat([df_team_records_22_to_24, df_2025], ignore_index=True)


,Year,TeamId,Team,Classification,Conference,Division,ExpectedWins,Total Games,Total Wins,Total Losses,Total Ties,Fpi,CoachingChange,Coach
0,2022,2000,Abilene Christian,fcs,Western Athletic,NaN,6.619276,11,7,4,0,NaN,False,NaN
4,2022,2005,Air Force,fbs,Mountain West,Mountain,10.945056,13,10,3,0,3.750,False,Troy Calhoun
8,2022,2006,Akron,fbs,Mid-American,East,1.956927,12,2,10,0,-17.322,False,Joe Moorhead
12,2022,333,Alabama,fbs,SEC,West,10.457207,13,11,2,0,27.296,False,Nick Saban
16,2022,2010,Alabama A&M,fcs,SWAC,East,3.601764,11,4,7,0,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028,2022,275,Wisconsin,fbs,Big Ten,West,8.817187,13,7,6,0,8.148,True,Paul Chryst
1032,2022,2747,Wofford,fcs,Southern,NaN,2.684401,11,3,8,0,NaN,False,NaN
1036,2022,2751,Wyoming,fbs,Mountain West,Mountain,6.018762,13,7,6,0,-9.340,False,Craig Bohl
1040,2022,43,Yale,fcs,Ivy,NaN,6.001470,10,8,2,0,NaN,False,NaN
